In [ ]:
print("hello world")

hello world


In [ ]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 44.1 MB/s eta 0:00:00


In [ ]:
pwd

'/content'

In [ ]:
import os
from gensim.models import doc2vec
from gensim.models.doc2vec import TaggedDocument

In [ ]:
!pip install JPype1


In [ ]:
faqs = [
        ["1","결제는 어떻게 해야 되나요?","원하시는 상품의 등록자에게 다이렉트 메세지를 보내세요."],
        ["2","환불은 안되나요?","부분환불만 가능합니다.환불정책표를 살펴보세요."],
        ["3","할인은 없나요?",". 따로 없습니다."],
        ["4","만약 게시물 등록자의 이력이 허위일 경우 어떻게 하나요?","조사후,허위사실이 인정되면 전액환불해 드립니다."],
        ["5","제가 원하는 상품이 카테고리에 없을 경우 어떻게 하나요?","외주콜 게시판을 이용해 주세요."]
    ]

In [ ]:
import jpype
from konlpy.tag import Kkma

In [ ]:

kkma = Kkma()

def tokenizer_kkma(doc):
  jpype.attachThreadToJVM()
  token_doc = ['/'.join(word) for word in kkma.pos(doc)]
  return token_doc


In [ ]:
print(tokenizer_kkma(faqs[0][1]))

<ipython-input-9-97105d443696>:4: DeprecationWarning: jpype._core.attachThreadToJVM is deprecated, use java.lang.Thread.attach instead
  jpype.attachThreadToJVM()


['결제/NNG', '는/JX', '어떻/VA', '게/ECD', '하/VV', '어야/ECD', '되/VV', '나요/EFQ', '?/SF']


In [ ]:
#list 첫번째는 질문들 두번째는 인덱스(질문들 다 쪼개기)
token_faqs = [(tokenizer_kkma(row[1]),row[0]) for row in faqs]

tagged_faqs = [TaggedDocument(d,[c]) for d,c in token_faqs]

In [ ]:
token_faqs

[(['결제/NNG',
   '는/JX',
   '어떻/VA',
   '게/ECD',
   '하/VV',
   '어야/ECD',
   '되/VV',
   '나요/EFQ',
   '?/SF'],
  '1'),
 (['환불/NNG', '은/JX', '안되/VA', '나요/EFQ', '?/SF'], '2'),
 (['할인/NNG', '은/JX', '없/VA', '나요/EFQ', '?/SF'], '3'),
 (['만약/NNG',
   '게시물/NNG',
   '등록/NNG',
   '자/NNG',
   '의/JKG',
   '이력/NNG',
   '이/JKS',
   '허위/NNG',
   '이/VCP',
   'ㄹ/ETD',
   '경우/NNG',
   '어떻게/MAG',
   '하나/NNG',
   '이/VCP',
   '요/EFN',
   '?/SF'],
  '4'),
 (['저/NP',
   '가/JKS',
   '원하/VV',
   '는/ETD',
   '상품/NNG',
   '이/JKS',
   '카테고리/NNG',
   '에/JKM',
   '없/VA',
   '을/ETD',
   '경우/NNG',
   '어떻게/MAG',
   '하나/NNG',
   '이/VCP',
   '요/EFN',
   '?/SF'],
  '5')]

In [ ]:
tagged_faqs #태그값으로 구분하기 때문에 태그값을 꼭 넣어줘야 된다

[TaggedDocument(words=['결제/NNG', '는/JX', '어떻/VA', '게/ECD', '하/VV', '어야/ECD', '되/VV', '나요/EFQ', '?/SF'], tags=['1']),
 TaggedDocument(words=['환불/NNG', '은/JX', '안되/VA', '나요/EFQ', '?/SF'], tags=['2']),
 TaggedDocument(words=['할인/NNG', '은/JX', '없/VA', '나요/EFQ', '?/SF'], tags=['3']),
 TaggedDocument(words=['만약/NNG', '게시물/NNG', '등록/NNG', '자/NNG', '의/JKG', '이력/NNG', '이/JKS', '허위/NNG', '이/VCP', 'ㄹ/ETD', '경우/NNG', '어떻게/MAG', '하나/NNG', '이/VCP', '요/EFN', '?/SF'], tags=['4']),
 TaggedDocument(words=['저/NP', '가/JKS', '원하/VV', '는/ETD', '상품/NNG', '이/JKS', '카테고리/NNG', '에/JKM', '없/VA', '을/ETD', '경우/NNG', '어떻게/MAG', '하나/NNG', '이/VCP', '요/EFN', '?/SF'], tags=['5'])]

In [ ]:
#모델 만들기
import multiprocessing
cores = multiprocessing.cpu_count()
d2v_faqs = doc2vec.Doc2Vec(
    vector_size = 50, ##
    alpha = 0.025,
    min_alpha = 0.025,
    hs =1,
    negative = 0, ##
    dm = 0,
    dbow_words = 1,
    min_count =0, ##
    workers = cores,
    seed = 0
)
d2v_faqs.build_vocab(tagged_faqs) # 단어사전 만들기

#학습시키기

for epoch in range(10):
  d2v_faqs.train(
      tagged_faqs,
      total_examples= d2v_faqs.corpus_count,
      epochs = d2v_faqs.epochs

  )

d2v_faqs.alpha -= 0.0025 #갈수록 학습률을 줄인다 => 속도향상을 위해
d2v_faqs.min_alpha = d2v_faqs.alpha #


In [ ]:
#들어온 질문을 백터화하기
test_string ="결제는 어떻게 하나요?"
token_test = tokenizer_kkma(test_string)
predict_vector = d2v_faqs.infer_vector(token_test)
predict_vector

array([ 1.20607438e-04, -4.43205377e-03,  7.50086829e-03, -1.01579390e-02,
        3.38605605e-06, -4.23811935e-03, -1.42869242e-02, -2.97254673e-03,
        1.24018015e-02,  8.97691399e-03,  1.46249998e-02,  6.40435843e-04,
        8.90326221e-04, -5.16678905e-03,  1.14264991e-02,  6.45197229e-04,
       -1.31245032e-02, -1.93658180e-03,  6.43060310e-03,  5.21113025e-03,
       -4.08595148e-03, -1.02869505e-02,  1.43586716e-03,  2.55509745e-04,
       -1.55134173e-03,  1.39704277e-03,  8.55676830e-03, -5.27267577e-03,
       -5.11871325e-03,  1.32161297e-03, -3.34083964e-03, -2.45687668e-03,
        9.40433238e-03,  1.44993188e-02, -6.52303733e-03,  1.12239541e-02,
        4.66202153e-03,  8.50292202e-03,  3.79789458e-03,  5.52091980e-03,
        7.55393784e-03,  6.31134491e-03,  5.98562066e-04, -9.28899925e-03,
       -2.42132926e-04, -2.11913418e-03, -4.01182473e-03, -4.22273343e-03,
       -1.48755289e-03,  6.56309677e-03], dtype=float32)

In [ ]:
#가장 유사한 값을 찾자
d2v_faqs.docvecs.most_similar(predict_vector,topn=2)

<ipython-input-39-4c0f47befb84>:2: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  d2v_faqs.docvecs.most_similar([predict_vector],topn=2)


[('5', 0.6130582094192505), ('1', 0.5998671650886536)]